In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm, animation, rc
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML, Image

%matplotlib inline

In [2]:
rc('animation', html='html5')
!brew install imagemagick

To reinstall 7.0.8-11, run `brew reinstall imagemagick`


### Algorithm

Objective: Find $argmin$ $f(x)$ where $f: \mathbb{R^n} \mapsto \mathbb{R}$.

**Initialize:** $x^0, \epsilon, \alpha$

While $|x_{n} - x_{n-1}| > \epsilon$ :

Update $x_n = x_{n-1} - \alpha \nabla f(x_{n-1})$

#### A simple example in one dimension

Find the minimum of $ y = x^2 $. 

$$\frac{dy}{dx} = 2x $$ 

$$x_n < x_{n-1} - \alpha 2x_{n}$$

In [3]:
%%capture 

# Define xrange, function, derivative 
x = np.arange(-5,5,.1)
y = lambda x: x ** 2   
dydx = lambda x: 2*x   

def gradient_descent(alpha, tolerance=None, iterations=None):
    # Initialize
    xvals = [x[0]] 
    yvals = [y(xvals[0])]
    xnew = xvals[-1] - alpha*dydx(xvals[-1])
    xcurrent = xvals[-1]
    
    if tolerance is not None:
        while abs(xcurrent - xnew) > tolerance:
            yvals.append(y(xcurrent))
            xvals.append(xcurrent)
            xcurrent = xnew
            xnew = xcurrent - alpha * dydx(xcurrent)
    else: 
        for _ in range(iterations):
            yvals.append(y(xcurrent))
            xvals.append(xcurrent)
            xcurrent = xnew
            xnew = xcurrent - alpha * dydx(xcurrent)
    return xvals, yvals

alpha =.1
tolerance = .01

xvals, yvals = gradient_descent(alpha, tolerance)

# Figure setup 
fig, ax = plt.subplots()
ax.plot(x,y(x), lw=2, label='y**2')

ax.set(xlim=(-5, 5), ylim=(-2, 30))
line, = ax.plot([],[], lw=1, marker='.', ms=5, mfc='red', c='red', label='estimation')
plt.legend()

In [4]:
def animate(i):
    line.set_data(xvals[:i], yvals[:i])
    
anim = animation.FuncAnimation(fig, animate, interval=200, frames=len(yvals), repeat=True)
anim.save('images/gradient-descent.gif', writer='imagemagick',fps=5)

In [5]:
Image(url='images/gradient-descent.gif')

If the learning rate $\alpha$ is set to be high, then the algorithm will take larger steps.

In [6]:
%%capture 

alpha = .8 
xvals, yvals = gradient_descent(alpha, iterations=50)

# Figure setup 
fig, ax = plt.subplots()
ax.plot(x,y(x), lw=2, label='y**2')

ax.set(xlim=(-5, 5), ylim=(-2, 30))
line, = ax.plot([],[], lw=1, marker='.', ms=5, mfc='red', c='red', label='estimation')
plt.legend()

In [7]:
anim = animation.FuncAnimation(fig, animate, interval=50, frames=len(yvals), repeat=True)
anim.save('images/gradient-descent-larger-alpha-2d.gif', writer='imagemagick',fps=1)

In [8]:
Image(url='images/gradient-descent-larger-alpha-2d.gif')

If the learning rate $\alpha$ is set too high the algorithm will diverge. 

In [9]:
%%capture 

alpha = 1 
xvals, yvals = gradient_descent(alpha, iterations=50)

# Figure setup 
fig, ax = plt.subplots()
ax.plot(x,y(x), lw=2, label='y**2')

ax.set(xlim=(-5, 5), ylim=(-2, 30))
line, = ax.plot([],[], lw=1, marker='.', ms=5, mfc='red', c='red', label='estimation')
plt.legend()

In [10]:
anim = animation.FuncAnimation(fig, animate, interval=100, frames=len(yvals), repeat=True)
anim.save('images/gradient-descent-largest-alpha-2d.gif', writer='imagemagick',fps=20)
Image(url='images/gradient-descent-largest-alpha-2d.gif')

### Two dimensional example

Let $f(x) = x^2 + y^2$. 

Calculative derivatives: 

$$\frac{df}{dx} = 2x$$

$$\frac{df}{dy} = 2y$$

In [11]:
%%capture
a = np.arange(-1,1,.1)
b = np.arange(-1,1,.1)
A, B = np.meshgrid(a,b)

f = lambda x,y: x**2 + y**2
C = f(A,B)
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.can_pan()
ax.plot_surface(A,B,C, alpha=.5)
line, = ax.plot([],[],[], lw=1, marker='.', ms=1, mfc='red', c='red', label='estimation')

In [12]:
dfdx = lambda x: 2*x
dfdy = lambda y: 2*y

alpha = .1
tolerance_a = .001
tolerance_b = .001

vals_a = [a[0]] 
vals_b = [b[0]]
vals_c = [f(vals_a[0], vals_b[0])]

new_a = vals_a[-1] - alpha*dfdx(vals_a[-1])
new_b = vals_b[-1] - alpha*dfdy(vals_b[-1])

current_a = vals_a[0]
current_b = vals_b[0]

while abs(current_a - new_a) > tolerance_a and abs(current_b - new_b) > tolerance_b:
    vals_c.append(f(current_a, current_b))
    vals_a.append(current_a)
    vals_b.append(current_b)
    
    current_a = new_a
    current_b = new_b
    
    new_a = current_a - alpha * dfdx(current_a)
    new_b = current_b - alpha * dfdx(current_b)

In [14]:
%matplotlib notebook
def animate(i):
    line.set_data(vals_a[:i], vals_b[:i])
    line.set_3d_properties(vals_c[:i])
    return line
    
anim = animation.FuncAnimation(fig, animate, interval=100, frames=len(vals_c), repeat=True)
anim.save('images/gradient-descent-3d.gif', writer='imagemagick', fps=5)
Image(url='images/gradient-descent-3d.gif')